In [2]:
import cv2
import torch
from ultralytics import YOLO
import carla
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load YOLOv5 model
model = YOLO("yolov8n.pt")

100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:04<00:00, 1.59MB/s]


In [4]:
client = carla.Client('localhost', 2000)

world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*cybertruck*')
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [5]:
# Camera Pos
CAMERA_POS_Z = 3
CAMERA_POS_X = 0


camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image' : np.zeros((image_h, image_w,4))}
# Open live stream from the camera
camera.listen(lambda image: camera_callback(image, camera_data))

In [8]:
vehicle.set_autopilot(True)

In [6]:
while True:
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break
    image = camera_data['image']
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_frame_3ch = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    # Performs inference on the current frame
    results = model(gray_frame_3ch)

    # Filter results for cars only)
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            if class_id == 2: # Class ID 2 is 'car' in COCO
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Get bounding box coordinates
                confidence = box.conf[0] # Confidence score
                # Draw the bouding vox aroung the detected car
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
                label = f'Car {confidence:.2f}'
                cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imshow('Canny camera', image)

cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


0: 384x640 (no detections), 172.4ms
Speed: 6.6ms preprocess, 172.4ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 107.6ms
Speed: 2.3ms preprocess, 107.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.1ms
Speed: 1.0ms preprocess, 64.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.3ms
Speed: 2.0ms preprocess, 65.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 71.1ms
Speed: 2.5ms preprocess, 71.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.5ms
Speed: 1.6ms preprocess, 56.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.1ms
Speed: 3.0ms preprocess, 62.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.4ms
Speed: 1.5ms preprocess, 46.4